# Get the Pre-Trained Model

In [ ]:
# clone the repo
!git clone https://github.com/Srijith-rkr/Whispering-LLaMA.git code

# Clone the weights and rename the folder as weights
!git lfs install
!mkdir weights
!wget -P weights/ https://huggingface.co/Srijith-rkr/Whispering-LLaMA/resolve/main/alpaca_a.pth
!wget -P weights/ https://huggingface.co/Srijith-rkr/Whispering-LLaMA/resolve/main/tokenizer.model
# !git clone https://huggingface.co/Srijith-rkr/Whispering-LLaMA

# You can also download the weights using the UI in HuggingFace from the repo 'https://huggingface.co/Srijith-rkr/Whispering-LLaMA' if you have issues with git lfs

# Refer to the READ ME file to build your environment

Cloning into 'code'...
remote: Enumerating objects: 242, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 242 (delta 44), reused 37 (delta 37), pack-reused 176 (from 1)
Receiving objects: 100% (242/242), 10.74 MiB | 11.58 MiB/s, done.
Resolving deltas: 100% (93/93), done.
Git LFS initialized.
--2024-10-28 12:41:18--  https://huggingface.co/Srijith-rkr/Whispering-LLaMA/resolve/main/alpaca_a.pth
Resolving huggingface.co (huggingface.co)... 3.165.160.61, 3.165.160.11, 3.165.160.12, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.61|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/57/fe/57fe57c6d17e96a78ef8cd228804abfdc148fd4a54af09cb7af16d14c45a6f79/e60dccf49a4aaaa4ffd87151ca2a52fbbfd9e962e0beda55306bdf3c00fe8807?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27alpaca_a.pth%3B+filename%3D%22alpaca_a.pth%22%3B&Expires=1730378478&Policy=eyJTdGF

In [ ]:
import torch

In [ ]:
# To merge the Alpaca weigts from the HuggingFace repo we just cloned
# a = torch.load('weights/alpaca_a.pth')
# b = torch.load('weights/alpaca_b.pth')
# c = torch.load('weights/alpaca_c.pth')

# merging
# alpaca_checkpoint = a|b|c
alpaca_checkpoint = torch.load('weights/alpaca_a.pth')

# saving
# torch.save(alpaca_checkpoint,'weights/alpaca.pth')

<ipython-input-3-ebd62394ea56>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  alpaca_checkpoint = torch.load('weights/alpaca_a.pth')


# Train Code

## Install Package

In [ ]:
!pip install -r code/requirementes.txt --q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install evaluate jiwer --q

In [ ]:
!pip install --q datasets[audio]

In [ ]:
!pip install --q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Getting the Dataset

In [ ]:
from google.colab import userdata
import os

HF_TOKEN = userdata.get('HF_TOKEN')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_TOKEN

In [ ]:
!huggingface-cli login --token $HUGGINGFACEHUB_API_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from datasets import load_dataset

dataset = load_dataset("lmms-lab/librispeech", "librispeech_dev_clean", revision='848ff3c79e7be380f9b8332ae0a8c569edac666e', split='librispeech_dev_clean[10:15]')
torch.save(dataset, "dataset_train.pt")

dataset = load_dataset("lmms-lab/librispeech", "librispeech_dev_clean", revision='848ff3c79e7be380f9b8332ae0a8c569edac666e', split='librispeech_dev_clean[20:25]')
torch.save(dataset, "dataset_test.pt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.73k [00:00<?, ?B/s]

(…)ispeech_dev_clean-00000-of-00002.parquet:   0%|          | 0.00/322M [00:00<?, ?B/s]

(…)ispeech_dev_clean-00001-of-00002.parquet:   0%|          | 0.00/290M [00:00<?, ?B/s]

Generating librispeech_dev_clean split:   0%|          | 0/2703 [00:00<?, ? examples/s]

## Train and Inference Model

In [ ]:
# Train model
!python code/training/WL-S.py --lr 1e-3 --d 1 --pretrained_path 'weights/alpaca_a.pth' --tokenizer_path 'weights/tokenizer.model' --data 'dataset'

/content/code/training/WL-S.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load(train_path,map_location=torch.device('cpu'))
/content/code/training/

In [ ]:
# To inference the model
!python code/Inference/WL-S.py --pretrained_path 'weights/alpaca_a.pth' --tokenizer_path 'weights/tokenizer.model' --data_path 'dataset_test.pt' --save_dir 'weights' --root 'code/Inference'

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
2024-10-28 12:49:21.373337: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-28 12:49:21.641435: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-28 12:49:21.712880: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-28 12:49:22.132548: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimiz